# Pandas and Time Series Data
## Exploring Open Power Systems

[Source of data](https://open-power-system-data.org) for daily consumption, wind, solar, wind+solar.

Credit:
- [https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas](https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas)
- [Ian Ozsvald - PyData 2019 talk](https://www.youtube.com/watch?v=8upGdZMlkYM&list=PLGVZCDnMOq0ocea1dd0it7jX7HgvZCjSW&index=13)

Data - [https://github.com/jenfly/opsd/raw/master/opsd_germany_daily.csv](https://github.com/jenfly/opsd/raw/master/opsd_germany_daily.csv)
        

In [ ]:
%load_ext watermark

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn

import watermark

In [ ]:
%matplotlib inline

In [ ]:
%watermark -a "Your Name" -u -d -v --iversions

## Date objects

Datatime objects are timezone aware or unaware. In general you cannot mix and match and it doesn't really make sense to try.

Generally better to have timezones attached to datetimes if you can.

Pandas has some helpful tools for creating and manupulating datetimes.

In [ ]:
pd.to_datetime("2018-01-26 15:17")

In [ ]:
pd.to_datetime("2018-01-26")

In [ ]:
pd.to_datetime("2021-03-01T14:04:37Z") # ISO 8601 is a good standard to use!

In [ ]:
pd.to_datetime("2018-01-26 15:17", utc=True)

In [ ]:
# Beware!

pd.to_datetime("8/7/1999")

In [ ]:
pd.to_datetime("8/7/1999", dayfirst=True)

We can also create date ranges using pandas!

In [ ]:
pd.date_range("2019-04-01", freq="1d", periods=45)

In [ ]:
pd.timedelta_range(start="1 day", end="14 day", periods=14)

In [ ]:
pd.timedelta_range(start="1 day", periods=14, freq="12H")

# Loading Open Utilities Data

Pandas can read from many sources including CSV, SQL, HDF5 and more ... (read the documentation)

In [ ]:
opsd_daily_df = pd.read_csv("https://github.com/jenfly/opsd/raw/master/opsd_germany_daily.csv")

In [ ]:
opsd_daily_df.shape

In [ ]:
opsd_daily_df.head()

In [ ]:
opsd_daily_df.tail()

In [ ]:
opsd_daily_df.dtypes

In [ ]:
opsd_daily_df.loc[:, "Date"] = opsd_daily_df['Date'].map(pd.to_datetime)

In [ ]:
opsd_daily_df = opsd_daily_df.set_index('Date')

In [ ]:
opsd_daily_df.sample(5)

In [ ]:
opsd_daily_df.info()

### Enriching the data with new columns

In [ ]:
opsd_daily_df["Year"] = opsd_daily_df.index.year
opsd_daily_df["Month"] = opsd_daily_df.index.month
opsd_daily_df["Weekday Name"] = opsd_daily_df.index.day_name()
opsd_daily_df["Weekday"] = opsd_daily_df.index.weekday

opsd_daily_df["Is Weekend"] = opsd_daily_df["Weekday"].isin((5,6))

In [ ]:
opsd_daily_df.head(7)

In [ ]:
opsd_daily_df["Weekday Name"].value_counts()

In [ ]:
opsd_daily_df.describe()

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
profile = ProfileReport(opsd_daily_df, title="Pandas Profiling Report")

In [ ]:
profile

## Visualisation

In [ ]:
import seaborn as sns

In [ ]:
# Use default style for Seaborn and set plot size

sns.set(rc={'figure.figsize': (16, 8)})

In [ ]:
opsd_daily_df['Consumption'].plot(linewidth=0.5)

In [ ]:
cols_plot = ["Consumption", "Solar", "Wind"]
axes = opsd_daily_df[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', subplots=True)
for ax in axes:
    ax.set_ylabel("Daily Totals (GWh)")

#### What behaviours can you see in the data?

What patterns are identifiable by eye?

### Exploring the patterns

In [ ]:
ax = opsd_daily_df.loc["2017", "Consumption"].plot()
ax.set_ylabel("Daily Consumption (GWh)")
ax.set_title("2017's Consumption by Day")

### Exercise: Can you group the data to get monthly means for consumption?

- Take a subset of the data for 2017
- Use the ```groupby``` function to aggregate the data by month
- You can calculate the mean, but you can also use other aggrgate functions e.g. max, min, sum, median, std. Experiment and see what you find
- Don't forget to plot the results!

In [ ]:
# Code Here

In [ ]:
# Code here

In [ ]:
# Plot here

### Diving in deeper

In [ ]:
ax = opsd_daily_df.loc["2017-01":"2017-03", "Consumption"].plot(marker='o', linestyle='-')
ax.set_ylabel("Daily Consumption (GWh)");

In [ ]:
import matplotlib.dates as mdates

In [ ]:
fig, ax = plt.subplots()
ax.plot(opsd_daily_df.loc["2017-01": "2017-03", "Consumption"], marker='o', linestyle='-')
ax.set_ylabel("Daily Consumption (GWh)")
ax.set_title("Jan-Mar 2017 Electricity Consumption")

# Change the markers to weekly intervals
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MONDAY))

# Format the x-tick labels
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'));

### Grouping on the weekday to see the mean behaviour by day of week

In [ ]:
mean_by_weekday_df = opsd_daily_df.loc["2017", ["Consumption", "Weekday"]].groupby("Weekday").mean()
mean_by_weekday_df

In [ ]:
mean_by_weekday_df.plot();

In [ ]:
day_of_week = {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thursday", 4: "Friday", 5: "Saturday", 6: "Sunday"}
new_index = mean_by_weekday_df.index.map(day_of_week)
mean_by_weekday_df.set_index(new_index)

In [ ]:
mean_by_weekday_df.set_index(new_index).plot();

## Summarising the seasonality using Seaborn

Looking for patterns, behaviours and outliers

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
for name, ax in zip(["Consumption", "Solar", "Wind"], axes):
    sns.boxplot(data=opsd_daily_df, x="Month", y=name, ax=ax)
    ax.set_ylabel("GWh")
    ax.set_title(f"Plot of {name}")
    
    # Clean up the automatic x-axis label from all but the last plot
    if ax != axes[-1]:
        ax.set_xlabel("")

In [ ]:
sns.boxplot(data=opsd_daily_df, x="Weekday Name", y="Consumption");

#### What if we want the data in a different order?

In [ ]:
day_of_week

In [ ]:
day_of_week.values()

In [ ]:
sns.boxplot(data=opsd_daily_df, x="Weekday Name", y="Consumption", order=day_of_week.values())

#### Where do the outliers come from? Are there any holidays in the data? 

In [ ]:
daily_mask = opsd_daily_df['Weekday Name'] == "Monday"
opsd_daily_df[daily_mask].query("Consumption < 1000").sort_values("Month")

## Autocorrelation and lags

How simialr is today's point to the same point N days in the future? Autocorreclation tests all frequencies.

Lags look at 1 frequency (default is 1 unit ahead)

In [ ]:
from pandas.plotting import autocorrelation_plot

In [ ]:
autocorrelation_plot(opsd_daily_df["Consumption"])

In [ ]:
fig, ax = plt.subplots();
autocorrelation_plot(opsd_daily_df["Consumption"], ax=ax)
ax.set_xlim(0, 30)

In [ ]:
fig, ax = plt.subplots();
autocorrelation_plot(opsd_daily_df["Consumption"], ax=ax)
ax.set_xlim(0, 360)

Lag plot shows structure between $y(t)$ and $y(t+1)$. A visual relationship suggests there is structure in the data.

In [ ]:
from pandas.plotting import lag_plot

data = opsd_daily_df.loc["2013"]
lag_plot(data["Consumption"]);

In [ ]:
lag_plot(data["Consumption"], c=data["Is Weekend"][:-1], cmap='viridis');

#### Exercise: Can you show the lag plot by day of week?

The Viridis colourmap goes from purple to green to yellow - do we see any daily structure?

In [ ]:
# your code here

## Data ranges and resampling

In [ ]:
pd.date_range("2020-02-04", periods=12, freq="H")

In [ ]:
opsd_daily_df.index[:5]

Let's make a small copy

In [ ]:
times_sample = pd.to_datetime(["2013-02-03", "2013-02-06", "2013-02-08"])

consum_sample_df = opsd_daily_df.loc[times_sample, ["Consumption"]].copy()
consum_sample_df

In [ ]:
# Convert the data to daily frequency, without filling in any missing data
consum_freq_df = consum_sample_df.asfreq('D')
consum_freq_df

### How do we fill in the missing data?

In [ ]:
consum_freq_df["Consumption - Forward Fill"] = consum_sample_df.asfreq("D", method="ffill")
consum_freq_df["Consumption - Backward Fill"] = consum_sample_df.asfreq("D", method="bfill")
consum_freq_df

### Weekly resampling - downsampling our data

In [ ]:
# Specify the columns we want to include
data_columns = ["Consumption", "Wind", "Solar", "Wind+Solar"]

# Resample to weekly frequency, aggregating with mean
opsd_weekly_mean_df = opsd_daily_df[data_columns].resample('W').mean()
opsd_weekly_mean_df.tail(10)

What's going on? What do we expect to see?

In [ ]:
start, end = "2017-01", "2017-06"

# Plot daily and weekly resample time series together
fig, ax = plt.subplots()
ax.plot(opsd_daily_df.loc[start:end, "Solar"],
        marker='.', 
        linestyle="-", linewidth=0.5, 
        label="Daily"
       )

ax.plot(opsd_weekly_mean_df.loc[start:end, "Solar"],
        marker="o", markersize=8,
        linestyle="-",
        label="Weekly Mean Resample"
       )

ax.set_ylabel("Solar Production (GWh)")
ax.legend()

### Rolling means

In [ ]:
# Calculate the centred 7-day rolling mean (not centred, using history only)

opsd_7d_df = opsd_daily_df[data_columns].rolling(7, center=False).mean()
opsd_7d_df.head(10)

In [ ]:
start, end = "2017-01", "2017-06"

# Plot daily and weekly resample time series together
fig, ax = plt.subplots()
ax.plot(opsd_daily_df.loc[start:end, "Solar"],
        marker='.', 
        linestyle="-", linewidth=0.5, 
        label="Daily"
       )

ax.plot(opsd_weekly_mean_df.loc[start:end, "Solar"],
        marker="o", markersize=8,
        linestyle="-",
        label="Weekly Mean Resample"
       )

ax.plot(opsd_7d_df.loc[start:end, "Solar"],
        marker=".",
        linestyle="-",
        label="7d Rolling Mean"
       )

ax.set_ylabel("Solar Production (GWh)")
ax.legend()

### Trends

If we plot 365 trend vs a 7 day trend what can we see for overall consumption and green energy production

In [ ]:
opsd_365d_df = opsd_daily_df[data_columns].rolling(window=365, center=False, min_periods=360).mean()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(opsd_daily_df.loc[:, "Consumption"],
        marker='.', markersize=2,
        linestyle="None", color="0.6",
        label="Daily"
       )

ax.plot(opsd_7d_df.loc[:, "Consumption"],
        linewidth=2,
        label="7d Rolling Mean"
       )

ax.plot(opsd_365d_df.loc[:, "Consumption"],
        linewidth=3, color="0.2",
        label="Trend (365d Rolling Mean)"
       )

ax.xaxis.set_major_locator(mdates.YearLocator())
ax.legend()
ax.set_xlabel("Year")
ax.set_ylabel("Consumption (GWh)")
ax.set_title("Treads in Electricity Consumption")

In [ ]:
fig, ax = plt.subplots()

for power_source in ["Wind", "Solar", "Wind+Solar"]:
    ax.plot(opsd_365d_df.loc[:, power_source],
            linewidth=2,
            label=f"{power_source}"
           )
    
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.set_ylim(0, 400)    
ax.legend()
ax.set_xlabel("Year")
ax.set_ylabel("Production (GWh)")
ax.set_title("Treads in Electricity Production (365d Rolling Mean)");

## What is the share of green power over the years?

In [ ]:
# Compute the annual sums, setting the value to NaN for any year
# with less than 360 days of data

opsd_annual_df = opsd_daily_df[data_columns].resample("A").sum(min_count=360)

# The default index of the resampled data is the last day of each year
# Reset the index to only be the year component
opsd_annual_df = opsd_annual_df.set_index(opsd_annual_df.index.year)
opsd_annual_df.index.name = "Year"
opsd_annual_df

### Exercise: Compute the fraction of wind+solar to overall consumption and plot it

Make sure to label your plot appropriately

In [ ]:
# Code here

# Extension

Check out the Prophet time series modelling library from Facebook [here](https://facebook.github.io/prophet/)